In [1]:
# import matplotlib.pyplot as plt
# import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

from data_loader import data_label_split
from data_loader import generate_data_set
from data_loader import dmso_taxol_ProfileBag

from torch_exp import train
from torch_exp import test
from torch_exp import mini_noise_signal_cv


# from sklearn.model_selection import StratifiedKFold

import torch
import torch.utils.data as D 
import torch.nn as nn
# from torch.autograd import Variable
import torch.optim as optim

In [2]:
# drop_NA_data = pd.read_csv("moa_data_drop_NA.csv", index_col=0)

In [3]:
mini_data = pd.read_csv("mini_moa_data_drop_NA.csv",index_col=0)

In [4]:
sf_drop_NA_data = mini_data[["compound", "concentration",
                                "moa", "row ID", "Iteration (#2)", "COND",
                               "AreaShape_Area_Nuclei", "AreaShape_Compactness_Nuclei"]]
bagData = dmso_taxol_ProfileBag(sf_drop_NA_data, 200, 50, 5, 0.5, "taxol", "DMSO", 0.5)
dataloader = D.DataLoader(bagData, batch_size=1, shuffle=True)

In [5]:
# data, label = iter(dataloader).next()

# Deep Set Model

In [6]:
class SmallDeepSet(nn.Module):
    def __init__(self, pool="max"):
        super().__init__()
        self.enc = nn.Sequential(
            nn.Linear(in_features=481, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=64),
        )
        self.dec = nn.Sequential(
            nn.Linear(in_features=64, out_features=32),
            nn.ReLU(),
            nn.Linear(in_features=32, out_features=1),
            nn.Sigmoid()
        )
        self.pool = pool

    def forward(self, x):
        x = self.enc(x)
        if self.pool == "max":
            x = x.max(dim=1)[0]
        elif self.pool == "mean":
            x = x.mean(dim=1)
        elif self.pool == "sum":
            x = x.sum(dim=1)
        elif self.pool == "min":
            x = x.min(dim=1)[0]
        x = self.dec(x)
        return x, torch.ge(x, 0.5)

In [7]:
# full_deepset = SmallDeepSet().cuda()
optimizer = optim.Adam(full_deepset.parameters(), lr=0.0005, betas=(0.9, 0.999), weight_decay=10e-5)

In [8]:
sf_deepset = SmallDeepSet().cuda()

In [9]:
# results = mini_noise_signal_cv(drop_NA_data, 1000, 100, 10, "taxol", "DMSO", full_deepset, 10, 50)

In [10]:
sf_results = mini_noise_signal_cv(sf_drop_NA_data, 10, 20, 1, "taxol", "DMSO", sf_deepset,optimizer, 3, 1)

training at different percent:   0%|          | 0/1 [00:00<?, ?it/s]
3 fold cross validation: 0it [00:00, ?it/s]

For 1 epoch: 0it [00:00, ?it/s]

For 1 epoch: 10it [00:00, 31.11it/s][A

3 fold cross validation: 1it [00:00,  2.36it/s]

For 1 epoch: 10it [00:00, 115.10it/s]

3 fold cross validation: 2it [00:00,  2.83it/s]

For 1 epoch: 10it [00:00, 124.29it/s]

3 fold cross validation: 3it [00:00,  3.87it/s]
training at different percent: 100%|██████████| 1/1 [00:00<00:00,  1.27it/s]


In [10]:
results

NameError: name 'results' is not defined

In [246]:
acc_control, acc_treat, pred_score_control, pred_score_treat = test(sf_deepset, dataloader)

In [253]:
(mean_control_accuracy, std_control_accuracy, 
            mean_treat_accuracy, std_treat_accuracy, 
            mean_pred_score_control, std_pred_score_control,
            mean_pred_score_treatment, std_pred_score_treatment) = sf_results

In [254]:
mean_pred_score_control

[0.7222758662762742]

In [255]:
sf_results

([0.29333333333333333],
 [0.4552898954390366],
 [0.7333333333333333],
 [0.4422166387140533],
 [0.7222758662762742],
 [0.37455220519738436],
 [0.7307330158321808],
 [0.3683108040093659])

In [136]:
full_deepset(x.float().cuda())

(tensor([[6.2371e-12]], device='cuda:0', grad_fn=<SigmoidBackward>),
 tensor([[0]], device='cuda:0', dtype=torch.uint8))

In [38]:
treatment_data = drop_NA_data[drop_NA_data["compound"] == treatment].reset_index(drop=True)
control_data = drop_NA_data[drop_NA_data["compound"] == control].reset_index(drop=True)
num_treat = int(size * perc)
# return treatment_data.sample(num_treat).append(control_data.sample(size - num_treat)).sample(frac = 1).reset_index(drop=True)

In [191]:
treatment_length = len(treatment_data)
control_length = len(control_data)

In [201]:
treat_size = (bag_size * perc).astype(int)
control_size = bag_size - treat_size

In [206]:
treat_length = 20
control_length = 22